In [ ]:
access_tokens = []
api_keys = []

In [ ]:
from kiteconnect import KiteConnect
for api_key, access_token in zip(api_keys, access_tokens):
    # Initialize KiteConnect instance
    kite = KiteConnect(api_key)
    kite.set_access_token(access_token)
    
    #place buy or sell order

In [ ]:
def buy_place_order_mkt(symbol, order_quantity):    
    order_id = kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=kite.EXCHANGE_NSE,
        tradingsymbol=symbol,
        quantity=order_quantity,
        order_type=kite.ORDER_TYPE_MARKET,
        product=kite.PRODUCT_MIS,
        transaction_type=kite.TRANSACTION_TYPE_BUY
    )


    print("Buy order placed successfully. Order ID:", order_id)

In [ ]:
def sell_place_order_mkt(symbol, order_quantity):    
    order_id = kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=kite.EXCHANGE_NSE,
        tradingsymbol=symbol,
        quantity=order_quantity,
        order_type=kite.ORDER_TYPE_MARKET,
        product=kite.PRODUCT_MIS,
        transaction_type=kite.TRANSACTION_TYPE_SELL
    )


    print("Order placed successfully. Order ID:", order_id)

### Three types of orders

#### Limit order

In [ ]:
def sell_place_order_lim(symbol, order_quantity, trigger_price):    
    order_id = kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=kite.EXCHANGE_NSE,
        tradingsymbol=symbol,
        quantity=order_quantity,
        order_type=kite.ORDER_TYPE_LIMIT,
        product=kite.PRODUCT_MIS,
        trigger_price = trigger_price,
        transaction_type=kite.TRANSACTION_TYPE_SELL
    )


    print("Order placed successfully. Order ID:", order_id)

def buy_place_order_lim(symbol, order_quantity, trigger_price):    
    order_id = kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=kite.EXCHANGE_NSE,
        tradingsymbol=symbol,
        quantity=order_quantity,
        order_type=kite.ORDER_TYPE_LIMIT,
        product=kite.PRODUCT_MIS,
        trigger_price = trigger_price,
        transaction_type=kite.TRANSACTION_TYPE_BUY
    )


    print("Order placed successfully. Order ID:", order_id)


#### There are two types of Stop-Loss orders: 
#### SL order (Stop-Loss Limit) = Price + Trigger Price. 
#### and 
#### SL-M order (Stop-Loss Market) = Only Trigger Price.

##### Sl order-limit

In [ ]:
# here we have a price and a trigger price.
order_id = kite.place_order(
    variety='regular',
    exchange=exchange,
    tradingsymbol=symbol,
    transaction_type=transaction_type,
    quantity=quantity,
    order_type=kite.ORDER_TYPE_SL,
    price=price,
    trigger_price=trigger_price,
    validity='DAY',  # DAY or IOC
    transaction_type=kite.TRANSACTION_TYPE_SELL, #sell for buy position and vice-versa
    product=kite.PRODUCT_MIS  # CNC for delivery-based orders, MIS for intraday
)

print(f"SL Order placed successfully. Order ID: {order_id}")

##### SL market

In [ ]:
# we basically put the price itm and make the order be placed instantly
order_id = kite.place_order(
    variety='regular',
    exchange=exchange,
    tradingsymbol=symbol,
    transaction_type=transaction_type,
    quantity=quantity,
    order_type=kite.ORDER_TYPE_SL,
    price=180, #assume market is 185, the order will be placed instantly.
    trigger_price=170,
    validity='DAY',  # DAY or IOC
    transaction_type=kite.TRANSACTION_TYPE_SELL, #sell for buy position and vice-versa
    product=kite.PRODUCT_MIS  # CNC for delivery-based orders, MIS for intraday
)

print(f"SL Order placed successfully. Order ID: {order_id}")

### Order with a trigger price and a stop loss. (cover order)

In [ ]:

try:
    target_order_id = None
    stoploss_order_id = None

    while True:
        ltp = kite.ltp(instruments)
        print(ltp)

        if ltp >= trigger_price and not target_order_id:
            # Place the target order for the target price
            buy_place_order_lim(symbol, order_quantity, trigger_price)
            target_order_id = 1
            
            print("Target order placed successfully. Order ID:", target_order_id)

        if ltp <= stop_loss and not stoploss_order_id:
            sell_place_order_lim(symbol, order_quantity, trigger_price)
            stoploss_order_id = 1
           
            print("Stop-loss order placed successfully. Order ID:", stoploss_order_id)

        if target_order_id or stoploss_order_id:
            # Exit the loop if either target or stop-loss order is placed
            break

    # Wait for the exit order to get executed
    # while True:
    #     order_status = kite.order_status(target_order_id) if target_order_id else kite.order_status(stoploss_order_id)

    #     if order_status["status"] == "COMPLETE":
    #         print("Exit order executed successfully.")
    #         break

except Exception as e:
    print("Order placement failed. Error message:", str(e))
